# Finding Spark

Whenever we work in Spark the first thing we need is the spark contect (sc).  We are going to use the module `findspark` to get access to the spark context.  First we need to install the module:

In [1]:
! pip install findspark

First we specify the path to spark - which for us is on the local VM:

In [25]:
import findspark
import os
findspark.init('/home/csumb/spark-1.6.0-bin-hadoop2.6')
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'

Now we can import pyspark and get the spark context:

In [26]:
import pyspark
try: 
    print(sc)
except NameError:
    sc = pyspark.SparkContext()
    print(sc)

# Creating an RDD

From the Spark documentation:

_"A Resilient Distributed Dataset (RDD), the basic abstraction in Spark, represents an immutable, partitioned collection of elements that can be operated on in parallel."_

_"Parallelized collections are created by calling SparkContext’s parallelize method on an existing iterable or collection in your driver program. The elements of the collection are copied to form a distributed dataset that can be operated on in parallel."_ 

For example, here is how to create a parallelized collection holding the numbers 1 to 5:


In [36]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
print(distData)

ParallelCollectionRDD[52] at parallelize at PythonRDD.scala:423


The RDD exists in the Spark Context which may or may not be in the notebook kernel.

We apply transformations and actions to the RDD. The RDD will execute operations in parallel, for example to add up elements of list.

Spark is heavily functional (built in Scala).  For example, map, reduce and filter operations are supported - these functions take functions or lambda functions as arguments: 

### Basics - transformations and actions

The RDD is not loaded in memory - it is just a pointer to the file.  Spark allows us to apply transformations to the RDD - but these are computed immediately - Spark is intentionally lazy.  Nothing is computed until we execute an action, at which point the Spark driver creates tasks to run on separate nodes in the Spark cluster.  Each node executes the transformations and actions and returns the results to the driver.   

http://vishnuviswanath.com/spark_rdd.html

In [16]:
distData = sc.parallelize(data) \
                .filter(lambda x : x > 3)
distData.collect()

[4, 5]

In [38]:
distData = sc.parallelize(data) \
                .filter(lambda x : x > 3) \
                .map(lambda x : x ** 2)
distData.collect()

[16, 25]

Actions collect the results:

In [44]:
from operator import add, mul 

distData = sc.parallelize(data) \
                .filter(lambda x : x > 3) \
                .map(lambda x : x ** 2) \
                .reduce(add)

distData

41

In [34]:

print('Sum : ' + distData.reduce(mul))
distData.reduce(mul)

120

### External Data Sources

We can also create RDDs from external data sources such as Hadoop, Amazon S3 and files. Here we will create a text file RDD.  NOte that we must use absolute paths since this code is pushed onto the Spark cluster - it is not run in the context of this notebook:

In [49]:
rdd = sc.textFile('/home/csumb/data-science-for-search/data/bike-item-titles.txt')
print(rdd)

rdd.take(10)

MapPartitionsRDD[72] at textFile at NativeMethodAccessorImpl.java:-2


[u'2,"ZIPP VUKA CARBON AERO BASE BAR AND EXTENSIONS COMPLETE TRIATHLON TT TRI CYCLING"',
 u'3,"Cycling Bicycle MTB Bike Fixie Gloss 3K Carbon Fiber Riser Bar Handlebar 31.8mm"',
 u'4,"BICYCLE RIMS 26""x 50MM RED 3 SPEED INTERNAL HUB WHEEL SET BEACH CRUISER BIKE"',
 u'5,"Mavic Crossride 26"" Mountain bike wheels and WTB Weirwolf Tires"',
 u'6,"New KCNC ARROW 7050 Alloy Stem ',
 u'7,"ROTOR QXL Aero Oval Road Chainring BCD110x5 53t"',
 u'8,"Yakima 4 pack SKS lock cores & 2 keys - A142 - roof rack locking cylinders"',
 u'9,"Sram Force Carbon Crank Gxp 110 Bcd No Chainrings 175 mm (2700)"',
 u'10,"THE ORIGINAL SQUIRT LONG LASTING DRY CHAIN BICYCLE LUBE WAX BASED"',
 u'11,"BV Bike Rear Saddle Bag Cycling Seat Post Pouch Bicycle Tail Storage NEW SB1-L"']

### Counting Words

To illustrate RDD basics, consider the simple program below which counts the number of words in the text file rdd we created earlier:

In [53]:
words_per_line = rdd.map(lambda s: len(s[0].split()))
#words_per_line.take(100)
total_words = words_per_line.reduce(add)
print(total_words)

9894


To reiterate - `words_per_line` is applies a transformation to the rdd - it is lazy and not evaluated until we apply an action - such as `reduce()`.  We can inspect the transformations applied to the RDD using the `toDebugString()` method:

In [54]:
print(words_per_line.toDebugString())

(1) PythonRDD[78] at RDD at PythonRDD.scala:43 []
 |  MapPartitionsRDD[72] at textFile at NativeMethodAccessorImpl.java:-2 []
 |  /home/csumb/data-science-for-search/data/bike-item-titles.txt HadoopRDD[71] at textFile at NativeMethodAccessorImpl.java:-2 []


### Term frequency in Spark

Many ways to do this - here are two:

In [77]:
terms1 = rdd.flatMap(lambda s : s.split(' ')) \
            .countByValue()

terms2 = rdd.flatMap(lambda s : s.split()) \
            .map(lambda w : (w, 1)) \
            .reduceByKey(lambda x,y : x+y) \
            .collectAsMap()

print(terms1['bike'])
print(terms2['bike'])
    

175
175


### Spark DataFrames API

If you look carefully above the text file is represented as a CSV and we did not parse the lines correctly.  CSV parsing is complex - but is made easier using Spark Data Frames which is an abstraction on top of RDDs. 

In [149]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv') \
        .options(header='false', inferSchema='true') \
        .load('/home/csumb/data-science-for-search/data/bike-item-titles.txt') \
        .selectExpr("C0 as id","C1 as item_title")

df.show()

+---+--------------------+
| id|          item_title|
+---+--------------------+
|  2|ZIPP VUKA CARBON ...|
|  3|Cycling Bicycle M...|
|  4|BICYCLE RIMS 26"x...|
|  5|Mavic Crossride 2...|
|  7|ROTOR QXL Aero Ov...|
|  8|Yakima 4 pack SKS...|
|  9|Sram Force Carbon...|
| 10|THE ORIGINAL SQUI...|
| 11|BV Bike Rear Sadd...|
| 12|HELIX BMX ROUND D...|
| 13|Waterproof Bicycl...|
| 14|Brand New CycleOp...|
| 15|Planet Bike LED S...|
| 16|Bike Bicycle Head...|
| 17|New Helmet Teenag...|
| 18|2 Pcs Bike Roller...|
| 19|FSA BICYCLE COMPR...|
| 20|Kenda Tube 26 X1....|
| 21|Bicycle Lock Set ...|
| 22|NEW DT Swiss 350 ...|
+---+--------------------+
only showing top 20 rows



In [150]:
df.schema

StructType(List(StructField(id,IntegerType,true),StructField(item_title,StringType,true)))

DataFrames API has functional model that can be applied to data frame objects:

In [151]:
df.filter(df['id'] >=5).filter(df['id'] <= 10)

#df.filter(df['id'] >=5).filter(df['id'] <= 10).count()

#df.filter(df['id'] >=5).filter(df['id'] <= 10).explain()

DataFrame[id: int, item_title: string]

Also has SQL interface:

In [152]:
sqlContext.registerDataFrameAsTable(df,'bikeitems')
sqlContext.tableNames()

sqlContext.sql("select id, item_title from bikeitems where id between 5 and 10").show()

+---+--------------------+
| id|          item_title|
+---+--------------------+
|  5|Mavic Crossride 2...|
|  7|ROTOR QXL Aero Ov...|
|  8|Yakima 4 pack SKS...|
|  9|Sram Force Carbon...|
| 10|THE ORIGINAL SQUI...|
+---+--------------------+



We can convert back and forth RDD <> DF:

In [187]:
rdd = df.rdd
df2 = rdd.toDF()

rdd.take(10)

[Row(id=2, item_title=u'ZIPP VUKA CARBON AERO BASE BAR AND EXTENSIONS COMPLETE TRIATHLON TT TRI CYCLING'),
 Row(id=3, item_title=u'Cycling Bicycle MTB Bike Fixie Gloss 3K Carbon Fiber Riser Bar Handlebar 31.8mm'),
 Row(id=4, item_title=u'BICYCLE RIMS 26"x 50MM RED 3 SPEED INTERNAL HUB WHEEL SET BEACH CRUISER BIKE'),
 Row(id=5, item_title=u'Mavic Crossride 26" Mountain bike wheels and WTB Weirwolf Tires'),
 Row(id=7, item_title=u'ROTOR QXL Aero Oval Road Chainring BCD110x5 53t'),
 Row(id=8, item_title=u'Yakima 4 pack SKS lock cores & 2 keys - A142 - roof rack locking cylinders'),
 Row(id=9, item_title=u'Sram Force Carbon Crank Gxp 110 Bcd No Chainrings 175 mm (2700)'),
 Row(id=10, item_title=u'THE ORIGINAL SQUIRT LONG LASTING DRY CHAIN BICYCLE LUBE WAX BASED'),
 Row(id=11, item_title=u'BV Bike Rear Saddle Bag Cycling Seat Post Pouch Bicycle Tail Storage NEW SB1-L'),
 Row(id=12, item_title=u'HELIX BMX ROUND DROPOUT SAVERS -FITS NEARLY ALL FRAMES -Fits 3/8" AND 10mm Axles')]

### Inverted Index

In [197]:
index = rdd.flatMap(lambda row : [ ( word, row[0]) for word in row[1].split(' ') ] ) 
index.take(10)

[(u'ZIPP', 2),
 (u'VUKA', 2),
 (u'CARBON', 2),
 (u'AERO', 2),
 (u'BASE', 2),
 (u'BAR', 2),
 (u'AND', 2),
 (u'EXTENSIONS', 2),
 (u'COMPLETE', 2),
 (u'TRIATHLON', 2)]

In [199]:
index = rdd.flatMap(lambda row : [ (word,  row[0]) for word in row[1].split(' ') ] ) \
            .groupByKey()
index.take(10)

[(u'', <pyspark.resultiterable.ResultIterable at 0x7f6df2a4d6d0>),
 (u'Powerlock-New', <pyspark.resultiterable.ResultIterable at 0x7f6df2a4d790>),
 (u'BLACK/SILVER', <pyspark.resultiterable.ResultIterable at 0x7f6df2a4df90>),
 (u'SecurityIng', <pyspark.resultiterable.ResultIterable at 0x7f6df2a4d050>),
 (u'SporstWear', <pyspark.resultiterable.ResultIterable at 0x7f6df2a4dc50>),
 (u'(28.6)', <pyspark.resultiterable.ResultIterable at 0x7f6df2a4de90>),
 (u'S-5', <pyspark.resultiterable.ResultIterable at 0x7f6df2a4dad0>),
 (u'Interloc', <pyspark.resultiterable.ResultIterable at 0x7f6df2a4d1d0>),
 (u'S-2', <pyspark.resultiterable.ResultIterable at 0x7f6df2a4d910>),
 (u'yellow', <pyspark.resultiterable.ResultIterable at 0x7f6df2a4d710>)]

In [206]:
index = rdd.flatMap(lambda row : [ (word,  row[0]) for word in row[1].split(' ') ] ) \
            .groupByKey() \
            .map(lambda x : (x[0], list(x[1])))
index.filter(lambda x : x[0] == 'Unicycle').collect()

[(u'Unicycle', [2138, 3748, 7232, 8777])]

In [207]:
index = rdd.flatMap(lambda row : [ (word,  row[0]) for word in row[1].split(' ') ] ) \
            .groupByKey() \
            .map(lambda x : (x[0], list(x[1]))).cache()

In [217]:
index.filter(lambda x : x[0] == 'Helmet').collect()

[(u'Helmet',
  [17,
   53,
   65,
   102,
   217,
   262,
   502,
   587,
   605,
   657,
   671,
   698,
   743,
   753,
   895,
   938,
   945,
   1077,
   1299,
   1460,
   1508,
   1651,
   1764,
   1897,
   1951,
   2132,
   2178,
   2368,
   2423,
   2434,
   2458,
   2514,
   2552,
   2630,
   2692,
   2825,
   2975,
   3079,
   3151,
   3189,
   3233,
   3233,
   3234,
   3265,
   3320,
   3484,
   3484,
   3485,
   3485,
   3667,
   3687,
   3990,
   4094,
   4094,
   4259,
   4681,
   4769,
   4808,
   4849,
   4899,
   5227,
   5241,
   5299,
   5478,
   5485,
   5550,
   5558,
   5577,
   5577,
   5599,
   5736,
   5803,
   5822,
   5890,
   6047,
   6047,
   6371,
   6375,
   6386,
   6386,
   6414,
   6418,
   6449,
   6549,
   6690,
   6725,
   6756,
   6940,
   7020,
   7032,
   7099,
   7190,
   7539,
   7721,
   7747,
   7757,
   7847,
   8055,
   8193,
   8239,
   8521,
   8664,
   8768,
   8793,
   9054,
   9137,
   9137,
   9428,
   9624,
   9753])]